In [ ]:
! pip install -r requirements.txt

In [ ]:
! pip install huggingface_hub

In [ ]:
import pandas as pd
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("ganga4364/garchen_rinpoche_inference_transcripts")

test_df = dataset["train"].to_pandas()
test_df.to_csv('test.csv', index=False)

In [ ]:
import os
import pandas as pd
from tqdm import tqdm
from transformers import pipeline
from datasets import Dataset, Audio
import logging
import requests

In [ ]:
import os
import pandas as pd
from tqdm import tqdm
from transformers import pipeline
from datasets import Dataset, Audio
import logging
import requests

# Configure logging
logging.basicConfig(filename='failed_downloads.log', level=logging.ERROR, format='%(asctime)s - %(message)s')

# Initialize generator pipelines
generator1 = pipeline(task="automatic-speech-recognition", model="ganga4364/mms_300_Garchen_Rinpoche-v5-base-Checkpoint-28000", device=0)
generator2 = pipeline(task="automatic-speech-recognition", model="ganga4364/mms_300_Garchen_Rinpoche-v6-ft-Checkpoint-25000", device=0)
generator3 = pipeline(task="automatic-speech-recognition", model="ganga4364/mms_300_Garchen_Rinpoche-v7-scracth-Checkpoint-23000", device=0)

# Function to download and validate audio
def download_audio(row):
    file_name = os.path.basename(row["url"])
    save_path = f"./downloads/{file_name}"
    os.makedirs(os.path.dirname(save_path), exist_ok=True)

    if os.path.exists(save_path):  # Skip if file exists
        return save_path

    try:
        response = requests.get(row["url"], timeout=10)
        response.raise_for_status()  # Check for HTTP errors
        with open(save_path, 'wb') as f:
            f.write(response.content)
        return save_path

    except Exception as e:
        logging.error(f"Failed to download {row['file_name']}: {e}")
        return None  # Return None if download failed

# Function to process inference in batches
def process_inference(batch):
    # Perform inference with generator1
    results1 = generator1(batch["path"])
    batch["ft_v5_base_inference_checkpoint-28000"] = [result["text"] for result in results1]

    # Perform inference with generator2
    results2 = generator2(batch["path"])
    batch["ft_v6_ft_inference_checkpoint-25000"] = [result["text"] for result in results2]

    # Perform inference with generator3
    results3 = generator3(batch["path"])
    batch["ft_v7_scratch_inference_checkpoint-23000"] = [result["text"] for result in results3]


    return batch


In [ ]:
#path to test file
input_file = "test.csv"
output_dir = "chunks_test"
output_file = "processed_test.csv"

os.makedirs(output_dir, exist_ok=True)

In [ ]:
! pip install pandarallel

In [ ]:

for i, chunk in enumerate(pd.read_csv(input_file, chunksize=1000), start=1):
    if chunk.empty:
        print(f"Chunk {i} is empty. Skipping.")
        continue

    chunk_file = os.path.join(output_dir, f"chunk_{i}.csv")
    if os.path.exists(chunk_file):
        print(f"Skipping chunk {i}, already processed.")
        continue

    tqdm.pandas(desc="Downloading audio files")
    from pandarallel import pandarallel
    # Initialize pandarallel (do this once at the start of your script)
    pandarallel.initialize(progress_bar=True, nb_workers=4)
    # Then replace your apply with parallel_apply
    chunk["path"] = chunk.parallel_apply(download_audio, axis=1)
    chunk = chunk[chunk["path"].notnull()]  # Remove rows with failed downloads

    # Reset index to avoid duplicate field errors
    chunk.reset_index(drop=True, inplace=True)

    # Convert to Dataset
    dataset = Dataset.from_pandas(chunk)
    #dataset = dataset.cast_column("path", Audio())

    # Perform batched inference
    dataset = dataset.map(process_inference, batched=True, batch_size=8)

    # Save processed chunk to CSV
    dataset.to_pandas().to_csv(chunk_file, index=False)
    print(f"Saved chunk {i} to {chunk_file}")


In [ ]:
# Save processed chunk to CSV
dataset.to_pandas().to_csv(chunk_file, index=False)
print(f"Saved chunk {i} to {chunk_file}")


In [ ]:

for i, chunk in enumerate(pd.read_csv(input_file, chunksize=1000), start=1):
    if chunk.empty:
        print(f"Chunk {i} is empty. Skipping.")
        continue

    chunk_file = os.path.join(output_dir, f"chunk_{i}.csv")
    if os.path.exists(chunk_file):
        print(f"Skipping chunk {i}, already processed.")
        continue

    tqdm.pandas(desc="Downloading audio files")
    chunk["path"] = chunk.progress_apply(download_audio, axis=1)
    chunk = chunk[chunk["path"].notnull()]  # Remove rows with failed downloads

    # Reset index to avoid duplicate field errors
    chunk.reset_index(drop=True, inplace=True)

    # Convert to Dataset
    dataset = Dataset.from_pandas(chunk)
    #dataset = dataset.cast_column("path", Audio())

    # Perform batched inference
    dataset = dataset.map(process_inference, batched=True, batch_size=8)

    # Save processed chunk to CSV
    dataset.to_pandas().to_csv(chunk_file, index=False)
    print(f"Saved chunk {i} to {chunk_file}")


In [ ]:
# Merge all chunk files into final output
all_chunks = [pd.read_csv(os.path.join(output_dir, f)) for f in sorted(os.listdir(output_dir)) if f.endswith(".csv")]
final_df = pd.concat(all_chunks, ignore_index=True)
final_df.to_csv(output_file, index=False)
print(f"All chunks merged and saved to {output_file}")


In [ ]:
final_df.head(2)

In [ ]:
! pip install pandas evaluate jiwer 
! pip install tibetan_wer 

In [ ]:
! pip install pandas

In [ ]:
! pip install evaluate

In [13]:
import pandas as pd
import numpy as np

from evaluate import load

cer_metric = load("cer")

# Load the CSV file
csv_file = "/home/gangagyatso/Downloads/stt-wav2vec2-single-speaker/merged_df.csv"  # Update with your actual file path
df = pd.read_csv(csv_file)

In [34]:
df.head(2)

,file_name,path,url,audio_duration,original_id,strata,age_group,duration_category,content_type,uni,...,ft_v8_scartch_inference_checkpoint-59000,ft_v8_scratch_cer_59000,ft_v8_scratch_lev_dist_59000,whisper-small-latin-added-tibetan-checkpoint-4000,whisper-small-tibetan-wylie-checkpoint-4000,whisper-small-tibetan-wylie-checkpoint-4000_to_tibetan,whisper-small-latin-added-tibetan-checkpoint-4000_cer,whisper-small-tibetan-wylie-checkpoint-4000_to_tibetan_cer,whisper-small-latin-added-tibetan-checkpoint-4000_lev_dist,whisper-small-tibetan-wylie-checkpoint-4000_to_tibetan_lev_dist
0,STT_GR_0001_0010_93600_to_106200,./downloads/STT_GR_0001_0010_93600_to_106200.wav,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...,12.600,STT_GR_0001,70-80__short__Teaching,70-80,short,Teaching,ཁོང་དེ་ནས་དགེ་འདུན་ཁྲི་ལ་སྔོན་ཟླ་རེ་བོད་གླེང་ད...,...,ཁོང་་དེ་ས་དགེ་འདུལ་ཁྲིི་ལང་བདག་རེད། བུ་གླིང་དེ...,0.290503,54,ཁོང་དེ་ནས་དགེ་དུས་འཕྲལ་པའི་ལྔ་ཟླ་རེད། དེ་བུ་གླ...,khong de nas dge du khri ba'i ngab da re yin ...,ཁོང་དེ་ནས་དགེ་དུ་ཁྲི་བའི་ངབ་ད་རེ་ཡིན་པོཨོན་གླི...,0.396648,0.374302,73,69
1,STT_GR_0033_0105_224320_to_225856,./downloads/STT_GR_0033_0105_224320_to_225856.wav,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...,1.536,STT_GR_0033,70-80__short__Teaching,70-80,short,Teaching,ཕ་གི་ཕར་རང་རེ་རྫས་ཚར་ལེ།,...,ཨ། ཕན་པ་ཡོད་ལ་དེ་ལྟ་ལུ།,0.750000,18,ཨ། ཕན་པ་རེད་གོ་ན་དེ་ལྟར་ནས།,phan phan ba lag gi yod na de lta bu la,ཕན་ཕན་བ་ལག་གི་ཡོད་ན་དེ་ལྟ་བུ་ལ,0.791667,0.916667,19,22


In [15]:
df.columns

Index(['file_name', 'path', 'url', 'audio_duration', 'original_id', 'strata',
       'age_group', 'duration_category', 'content_type', 'uni',
       'base_transcript', 'base_cer', 'base_lev_dist',
       'ft_v1_inference_checkpoint-5000', 'ft_v1_cer_5000',
       'ft_v1_lev_dist_5000', 'ft_v1_inference_checkpoint-10000',
       'ft_v1_cer_10000', 'ft_v1_lev_dist_10000',
       'ft_v1_inference_checkpoint-19000', 'ft_v1_cer_19000',
       'ft_v1_lev_dist_19000', 'ft_v2_inference_checkpoint-20000',
       'ft_v2_cer_20000', 'ft_v2_lev_dist_20000',
       'ft_v2_inference_checkpoint-32000', 'ft_v2_cer_32000',
       'ft_v2_lev_dist_32000', 'ft_v2_inference_checkpoint-43000',
       'ft_v2_cer_43000', 'ft_v2_lev_dist_43000',
       'ft_v3_inference_checkpoint-1000', 'ft_v3_cer_1000',
       'ft_v3_lev_dist_1000', 'ft_v3_inference_checkpoint-25000',
       'ft_v3_cer_25000', 'ft_v3_lev_dist_25000',
       'ft_v4_inference_checkpoint-22000', 'ft_v4_cer_22000',
       'ft_v4_lev_dist_22000', 

In [16]:
# Create a mapping of old column names to new column names
column_mapping = {
    'tibetan_transcript': 'whisper-small-latin-added-tibetan-checkpoint-4000',
    'wylie_transcript': 'whisper-small-tibetan-wylie-checkpoint-4000',
    'wylie_to_tibetan_text': 'whisper-small-tibetan-wylie-checkpoint-4000_to_tibetan'
}

# Apply the rename operation
df = df.rename(columns=column_mapping)

# Verify the columns have been renamed
print(df.columns.tolist())

['file_name', 'path', 'url', 'audio_duration', 'original_id', 'strata', 'age_group', 'duration_category', 'content_type', 'uni', 'base_transcript', 'base_cer', 'base_lev_dist', 'ft_v1_inference_checkpoint-5000', 'ft_v1_cer_5000', 'ft_v1_lev_dist_5000', 'ft_v1_inference_checkpoint-10000', 'ft_v1_cer_10000', 'ft_v1_lev_dist_10000', 'ft_v1_inference_checkpoint-19000', 'ft_v1_cer_19000', 'ft_v1_lev_dist_19000', 'ft_v2_inference_checkpoint-20000', 'ft_v2_cer_20000', 'ft_v2_lev_dist_20000', 'ft_v2_inference_checkpoint-32000', 'ft_v2_cer_32000', 'ft_v2_lev_dist_32000', 'ft_v2_inference_checkpoint-43000', 'ft_v2_cer_43000', 'ft_v2_lev_dist_43000', 'ft_v3_inference_checkpoint-1000', 'ft_v3_cer_1000', 'ft_v3_lev_dist_1000', 'ft_v3_inference_checkpoint-25000', 'ft_v3_cer_25000', 'ft_v3_lev_dist_25000', 'ft_v4_inference_checkpoint-22000', 'ft_v4_cer_22000', 'ft_v4_lev_dist_22000', 'ft_v5_base_inference_checkpoint-28000', 'ft_v5_base_cer_28000', 'ft_v5_base_lev_dist_28000', 'ft_v6_ft_inference_chec

In [17]:
import pandas as pd
import numpy as np

from evaluate import load

cer_metric = load("cer")

# Load the CSV file
#csv_file = "/home/gangagyatso/Downloads/stt-wav2vec2-single-speaker/merged_df.csv"  # Update with your actual file path
#df = pd.read_csv(csv_file)

# Define the inference columns and their corresponding CER column names
inference_to_cer_mapping = {
    "whisper-small-latin-added-tibetan-checkpoint-4000": "whisper-small-latin-added-tibetan-checkpoint-4000_cer",
    "whisper-small-tibetan-wylie-checkpoint-4000_to_tibetan": "whisper-small-tibetan-wylie-checkpoint-4000_to_tibetan_cer",
}

# Function to calculate CER
def calculate_cer(reference, prediction):
    try:
        if not reference or not prediction or pd.isna(reference) or pd.isna(prediction):
            return np.nan
        cer = cer_metric.compute(references=[reference], predictions=[prediction])
        return min(cer, 1.0)  # Cap at 1.0 as seen in your existing data
    except Exception as e:
        print(f"Error calculating CER: {e}")
        return np.nan

# Calculate and add CER columns
for inference_col, cer_col in inference_to_cer_mapping.items():
    print(f"Calculating {cer_col}...")
    df[cer_col] = df.apply(
        lambda row: calculate_cer(row['uni'], row[inference_col]), 
        axis=1
    )

# Save the updated DataFrame back to CSV
df.to_csv("processed_test_with_cer.csv", index=False)
print("CER calculations completed and saved to processed_test_with_cer.csv")

Calculating whisper-small-latin-added-tibetan-checkpoint-4000_cer...
Calculating whisper-small-tibetan-wylie-checkpoint-4000_to_tibetan_cer...
CER calculations completed and saved to processed_test_with_cer.csv


In [ ]:
! pip install python-Levenshtein

In [ ]:
! pip install jiwer

In [18]:
import pandas as pd
import numpy as np
from Levenshtein import distance
from evaluate import load

cer_metric = load("cer")

# Load the CSV file
csv_file = "processed_test_with_cer.csv"  # Update with your actual file path
df = pd.read_csv(csv_file)

# Define the inference columns and their corresponding CER column names
inference_to_cer_mapping = {
    "whisper-small-latin-added-tibetan-checkpoint-4000": "whisper-small-latin-added-tibetan-checkpoint-4000_lev_dist",
    "whisper-small-tibetan-wylie-checkpoint-4000_to_tibetan": "whisper-small-tibetan-wylie-checkpoint-4000_to_tibetan_lev_dist",

}

# Function to calculate CER
def calculate_lev_dist(reference, prediction):
    try:
        if not reference or not prediction or pd.isna(reference) or pd.isna(prediction):
            return np.nan
        lev_dist = distance(reference, prediction)
        return lev_dist
    except Exception as e:
        print(f"Error calculating CER: {e}")
        return np.nan

# Calculate and add CER columns
for inference_col, cer_col in inference_to_cer_mapping.items():
    print(f"Calculating {cer_col}...")
    df[cer_col] = df.apply(
        lambda row: calculate_lev_dist(row['uni'], row[inference_col]), 
        axis=1
    )

# Save the updated DataFrame back to CSV
df.to_csv("processed_test_with_cer_and_lev_dist.csv", index=False)  
print("CER calculations completed and saved to processed_test_with_cer_and_lev_dist.csv")

Calculating whisper-small-latin-added-tibetan-checkpoint-4000_lev_dist...
Calculating whisper-small-tibetan-wylie-checkpoint-4000_to_tibetan_lev_dist...
CER calculations completed and saved to processed_test_with_cer_and_lev_dist.csv


In [ ]:
! pip install python-Levenshtein

In [22]:
import pandas as pd 
df = pd.read_csv("processed_test_with_cer_and_lev_dist.csv")


In [23]:
df.columns

Index(['file_name', 'path', 'url', 'audio_duration', 'original_id', 'strata',
       'age_group', 'duration_category', 'content_type', 'uni',
       'base_transcript', 'base_cer', 'base_lev_dist',
       'ft_v1_inference_checkpoint-5000', 'ft_v1_cer_5000',
       'ft_v1_lev_dist_5000', 'ft_v1_inference_checkpoint-10000',
       'ft_v1_cer_10000', 'ft_v1_lev_dist_10000',
       'ft_v1_inference_checkpoint-19000', 'ft_v1_cer_19000',
       'ft_v1_lev_dist_19000', 'ft_v2_inference_checkpoint-20000',
       'ft_v2_cer_20000', 'ft_v2_lev_dist_20000',
       'ft_v2_inference_checkpoint-32000', 'ft_v2_cer_32000',
       'ft_v2_lev_dist_32000', 'ft_v2_inference_checkpoint-43000',
       'ft_v2_cer_43000', 'ft_v2_lev_dist_43000',
       'ft_v3_inference_checkpoint-1000', 'ft_v3_cer_1000',
       'ft_v3_lev_dist_1000', 'ft_v3_inference_checkpoint-25000',
       'ft_v3_cer_25000', 'ft_v3_lev_dist_25000',
       'ft_v4_inference_checkpoint-22000', 'ft_v4_cer_22000',
       'ft_v4_lev_dist_22000', 

In [24]:
print("base_cer mean:" ,df['base_cer'].mean())
print("ft_v1_cer_5000 mean:" ,df['ft_v1_cer_5000'].mean())
print("ft_v1_cer_10000 mean:" ,df['ft_v1_cer_10000'].mean())
print("ft_v1_cer_19000 mean:" ,df['ft_v1_cer_19000'].mean())
print("ft_v2_cer_20000 mean:" ,df['ft_v2_cer_20000'].mean())
print("ft_v2_cer_32000 mean:" ,df['ft_v2_cer_32000'].mean())
print("ft_v2_cer_43000 mean:" ,df['ft_v2_cer_43000'].mean())
print("ft_v3_cer_1000 mean:" ,df['ft_v3_cer_1000'].mean())
print("ft_v3_cer_25000 mean:" ,df['ft_v3_cer_25000'].mean())
print("ft_v4_cer_22000 mean:" ,df['ft_v4_cer_22000'].mean())
print("ft_v5_base_cer_28000 mean:" ,df['ft_v5_base_cer_28000'].mean())
print("ft_v6_ft_cer_25000 mean:" ,df['ft_v6_ft_cer_25000'].mean())
print("ft_v7_scratch_cer_23000 mean:" ,df['ft_v7_scratch_cer_23000'].mean())
print("ft_v8_scratch_cer_10000 mean:" ,df['ft_v8_scratch_cer_10000'].mean())
print("ft_v8_scratch_cer_55000 mean:" ,df['ft_v8_scratch_cer_55000'].mean())
print("ft_v8_scratch_cer_59000 mean:" ,df['ft_v8_scratch_cer_59000'].mean())
print("whisper-small-latin-added-tibetan-checkpoint-4000 cer mean:" ,df['whisper-small-latin-added-tibetan-checkpoint-4000_cer'].mean())
print("whisper-small-tibetan-wylie-checkpoint-4000_to_tibetan cer mean:" ,df['whisper-small-tibetan-wylie-checkpoint-4000_to_tibetan_cer'].mean())




base_cer mean: 0.2766951690814779
ft_v1_cer_5000 mean: 0.27408927238542324
ft_v1_cer_10000 mean: 0.23372037107449425
ft_v1_cer_19000 mean: 0.22933289935103854
ft_v2_cer_20000 mean: 0.22269392860735918
ft_v2_cer_32000 mean: 0.21756439369869338
ft_v2_cer_43000 mean: 0.2153139333090411
ft_v3_cer_1000 mean: 0.2336476706095409
ft_v3_cer_25000 mean: 0.22213554200186256
ft_v4_cer_22000 mean: 0.2266093220628319
ft_v5_base_cer_28000 mean: 0.21519361672096998
ft_v6_ft_cer_25000 mean: 0.22091044667156381
ft_v7_scratch_cer_23000 mean: 0.21855046572947306
ft_v8_scratch_cer_10000 mean: 0.24240847957505046
ft_v8_scratch_cer_55000 mean: 0.22025034419439046
ft_v8_scratch_cer_59000 mean: 0.23043649332091357
whisper-small-latin-added-tibetan-checkpoint-4000 cer mean: 0.29880807526513936
whisper-small-tibetan-wylie-checkpoint-4000_to_tibetan cer mean: 0.3848483478730869


In [25]:
print("base_lev_dist mean:" ,df['base_lev_dist'].mean())
print("ft_v1_lev_dist_5000 mean:" ,df['ft_v1_lev_dist_5000'].mean())
print("ft_v1_lev_dist_10000 mean:" ,df['ft_v1_lev_dist_10000'].mean())
print("ft_v1_lev_dist_19000 mean:" ,df['ft_v1_lev_dist_19000'].mean())
print("ft_v2_lev_dist_20000 mean:" ,df['ft_v2_lev_dist_20000'].mean())
print("ft_v2_lev_dist_32000 mean:" ,df['ft_v2_lev_dist_32000'].mean())
print("ft_v2_lev_dist_43000 mean:" ,df['ft_v2_lev_dist_43000'].mean())
print("ft_v3_lev_dist_1000 mean:" ,df['ft_v3_lev_dist_1000'].mean())
print("ft_v3_lev_dist_25000 mean:" ,df['ft_v3_lev_dist_25000'].mean())
print("ft_v4_lev_dist_22000 mean:" ,df['ft_v4_lev_dist_22000'].mean())
print("ft_v5_base_lev_dist_28000 mean:" ,df['ft_v5_base_lev_dist_28000'].mean())
print("ft_v6_ft_lev_dist_25000 mean:" ,df['ft_v6_ft_lev_dist_25000'].mean())
print("ft_v7_scratch_lev_dist_23000 mean:" ,df['ft_v7_scratch_lev_dist_23000'].mean())
print("ft_v8_scratch_lev_dist_10000 mean:" ,df['ft_v8_scratch_lev_dist_10000'].mean())
print("ft_v8_scratch_lev_dist_55000 mean:" ,df['ft_v8_scratch_lev_dist_55000'].mean())
print("ft_v8_scratch_lev_dist_59000 mean:" ,df['ft_v8_scratch_lev_dist_59000'].mean())
print("whisper-small-latin-added-tibetan-checkpoint-4000_lev_dist mean:" ,df['whisper-small-latin-added-tibetan-checkpoint-4000_lev_dist'].mean())
print("whisper-small-tibetan-wylie-checkpoint-4000_to_tibetan_lev_dist mean:" ,df['whisper-small-tibetan-wylie-checkpoint-4000_to_tibetan_lev_dist'].mean())



base_lev_dist mean: 19.17245240761478
ft_v1_lev_dist_5000 mean: 18.44120940649496
ft_v1_lev_dist_10000 mean: 15.65845464725644
ft_v1_lev_dist_19000 mean: 15.33370660694289
ft_v2_lev_dist_20000 mean: 14.883538633818588
ft_v2_lev_dist_32000 mean: 14.730123180291153
ft_v2_lev_dist_43000 mean: 14.608062709966406
ft_v3_lev_dist_1000 mean: 15.440089585666293
ft_v3_lev_dist_25000 mean: 14.991041433370661
ft_v4_lev_dist_22000 mean: 15.16909294512878
ft_v5_base_lev_dist_28000 mean: 14.96976483762598
ft_v6_ft_lev_dist_25000 mean: 15.213885778275476
ft_v7_scratch_lev_dist_23000 mean: 14.57110862262038
ft_v8_scratch_lev_dist_10000 mean: 16.829787234042552
ft_v8_scratch_lev_dist_55000 mean: 15.21612541993281
ft_v8_scratch_lev_dist_59000 mean: 15.858902575587907
whisper-small-latin-added-tibetan-checkpoint-4000_lev_dist mean: 23.25979843225084
whisper-small-tibetan-wylie-checkpoint-4000_to_tibetan_lev_dist mean: 29.949608062709967


In [26]:
print("base_cer std:" ,df['base_cer'].std())
print("ft_v1_cer_5000 std:" ,df['ft_v1_cer_5000'].std())
print("ft_v1_cer_10000 std:" ,df['ft_v1_cer_10000'].std())
print("ft_v1_cer_19000 std:" ,df['ft_v1_cer_19000'].std())
print("ft_v2_cer_20000 std:" ,df['ft_v2_cer_20000'].std())
print("ft_v2_cer_32000 std:" ,df['ft_v2_cer_32000'].std())
print("ft_v2_cer_43000 std:" ,df['ft_v2_cer_43000'].std())
print("ft_v3_cer_1000 std:" ,df['ft_v3_cer_1000'].std())
print("ft_v3_cer_25000 std:" ,df['ft_v3_cer_25000'].std())
print("ft_v4_cer_22000 std:" ,df['ft_v4_cer_22000'].std())
print("ft_v5_base_cer_28000 std:" ,df['ft_v5_base_cer_28000'].std())
print("ft_v6_ft_cer_25000 std:" ,df['ft_v6_ft_cer_25000'].std())
print("ft_v7_scratch_cer_23000 std:" ,df['ft_v7_scratch_cer_23000'].std())
print("ft_v8_scratch_cer_10000 std:" ,df['ft_v8_scratch_cer_10000'].std())
print("ft_v8_scratch_cer_55000 std:" ,df['ft_v8_scratch_cer_55000'].std())
print("ft_v8_scratch_cer_59000 std:" ,df['ft_v8_scratch_cer_59000'].std())
print("whisper-small-latin-added-tibetan-checkpoint-4000_cer std:" ,df['whisper-small-latin-added-tibetan-checkpoint-4000_cer'].std())
print("whisper-small-tibetan-wylie-checkpoint-4000_to_tibetan_cer std:" ,df['whisper-small-tibetan-wylie-checkpoint-4000_to_tibetan_cer'].std())




base_cer std: 0.16075142686673585
ft_v1_cer_5000 std: 0.18171980976826388
ft_v1_cer_10000 std: 0.17244792880666696
ft_v1_cer_19000 std: 0.17190517062534671
ft_v2_cer_20000 std: 0.17455029886076237
ft_v2_cer_32000 std: 0.1670877953428531
ft_v2_cer_43000 std: 0.16814154355068064
ft_v3_cer_1000 std: 0.17568876920450874
ft_v3_cer_25000 std: 0.17130500294831275
ft_v4_cer_22000 std: 0.1714633815167261
ft_v5_base_cer_28000 std: 0.16066051869804554
ft_v6_ft_cer_25000 std: 0.16997512240083704
ft_v7_scratch_cer_23000 std: 0.16717906434681518
ft_v8_scratch_cer_10000 std: 0.16544477870138555
ft_v8_scratch_cer_55000 std: 0.1637333213291196
ft_v8_scratch_cer_59000 std: 0.1647325278426431
whisper-small-latin-added-tibetan-checkpoint-4000_cer std: 0.17580701504497323
whisper-small-tibetan-wylie-checkpoint-4000_to_tibetan_cer std: 0.2005481872804231


In [27]:
print("base_lev_dist std:" ,df['base_lev_dist'].std())
print("ft_v1_lev_dist_5000 std:" ,df['ft_v1_lev_dist_5000'].std())
print("ft_v1_lev_dist_10000 std:" ,df['ft_v1_lev_dist_10000'].std())
print("ft_v1_lev_dist_19000 std:" ,df['ft_v1_lev_dist_19000'].std())
print("ft_v2_lev_dist_20000 std:" ,df['ft_v2_lev_dist_20000'].std())
print("ft_v2_lev_dist_32000 std:" ,df['ft_v2_lev_dist_32000'].std())
print("ft_v2_lev_dist_43000 std:" ,df['ft_v2_lev_dist_43000'].std())
print("ft_v3_lev_dist_1000 std:" ,df['ft_v3_lev_dist_1000'].std())
print("ft_v3_lev_dist_25000 std:" ,df['ft_v3_lev_dist_25000'].std())
print("ft_v4_lev_dist_22000 std:" ,df['ft_v4_lev_dist_22000'].std())
print("ft_v5_base_lev_dist_28000 std:" ,df['ft_v5_base_lev_dist_28000'].std())
print("ft_v6_ft_lev_dist_25000 std:" ,df['ft_v6_ft_lev_dist_25000'].std())
print("ft_v7_scratch_lev_dist_23000 std:" ,df['ft_v7_scratch_lev_dist_23000'].std())
print("ft_v8_scratch_lev_dist_10000 std:" ,df['ft_v8_scratch_lev_dist_10000'].std())
print("ft_v8_scratch_lev_dist_55000 std:" ,df['ft_v8_scratch_lev_dist_55000'].std())
print("ft_v8_scratch_lev_dist_59000 std:" ,df['ft_v8_scratch_lev_dist_59000'].std())
print("whisper-small-latin-added-tibetan-checkpoint-4000_lev_dist std:" ,df['whisper-small-latin-added-tibetan-checkpoint-4000_lev_dist'].std())
print("whisper-small-tibetan-wylie-checkpoint-4000_to_tibetan_lev_dist std:" ,df['whisper-small-tibetan-wylie-checkpoint-4000_to_tibetan_lev_dist'].std())


base_lev_dist std: 21.79365734265566
ft_v1_lev_dist_5000 std: 19.909328195806655
ft_v1_lev_dist_10000 std: 17.421535454982365
ft_v1_lev_dist_19000 std: 16.93500353273463
ft_v2_lev_dist_20000 std: 16.348704643421886
ft_v2_lev_dist_32000 std: 16.14383080904553
ft_v2_lev_dist_43000 std: 16.028660637297786
ft_v3_lev_dist_1000 std: 16.87683040373015
ft_v3_lev_dist_25000 std: 16.40497601460487
ft_v4_lev_dist_22000 std: 16.709717204794984
ft_v5_base_lev_dist_28000 std: 16.64493969786414
ft_v6_ft_lev_dist_25000 std: 17.197812061550955
ft_v7_scratch_lev_dist_23000 std: 15.466611464122776
ft_v8_scratch_lev_dist_10000 std: 19.463030600512184
ft_v8_scratch_lev_dist_55000 std: 17.316601064144933
ft_v8_scratch_lev_dist_59000 std: 18.459937889008685
whisper-small-latin-added-tibetan-checkpoint-4000_lev_dist std: 33.61533467141793
whisper-small-tibetan-wylie-checkpoint-4000_to_tibetan_lev_dist std: 43.62401440369452


In [28]:
print("base_cer min:" ,df['base_cer'].min())
print("ft_v1_cer_5000 min:" ,df['ft_v1_cer_5000'].min())
print("ft_v1_cer_10000 min:" ,df['ft_v1_cer_10000'].min())  
print("ft_v1_cer_19000 min:" ,df['ft_v1_cer_19000'].min())
print("ft_v2_cer_20000 min:" ,df['ft_v2_cer_20000'].min())
print("ft_v2_cer_32000 min:" ,df['ft_v2_cer_32000'].min())
print("ft_v2_cer_43000 min:" ,df['ft_v2_cer_43000'].min())
print("ft_v3_cer_1000 min:" ,df['ft_v3_cer_1000'].min())
print("ft_v3_cer_25000 min:" ,df['ft_v3_cer_25000'].min())
print("ft_v4_cer_22000 min:" ,df['ft_v4_cer_22000'].min())
print("ft_v5_base_cer_28000 min:" ,df['ft_v5_base_cer_28000'].min())
print("ft_v6_ft_cer_25000 min:" ,df['ft_v6_ft_cer_25000'].min())
print("ft_v7_scratch_cer_23000 min:" ,df['ft_v7_scratch_cer_23000'].min())
print("ft_v8_scratch_cer_10000 min:" ,df['ft_v8_scratch_cer_10000'].min())
print("ft_v8_scratch_cer_55000 min:" ,df['ft_v8_scratch_cer_55000'].min())
print("ft_v8_scratch_cer_59000 min:" ,df['ft_v8_scratch_cer_59000'].min())
print("whisper-small-latin-added-tibetan-checkpoint-4000_cer min:" ,df['whisper-small-latin-added-tibetan-checkpoint-4000_cer'].min())
print("whisper-small-tibetan-wylie-checkpoint-4000_to_tibetan_cer min:" ,df['whisper-small-tibetan-wylie-checkpoint-4000_to_tibetan_cer'].min())



base_cer min: 0.0
ft_v1_cer_5000 min: 0.0
ft_v1_cer_10000 min: 0.0
ft_v1_cer_19000 min: 0.0
ft_v2_cer_20000 min: 0.0
ft_v2_cer_32000 min: 0.0
ft_v2_cer_43000 min: 0.0
ft_v3_cer_1000 min: 0.0
ft_v3_cer_25000 min: 0.0
ft_v4_cer_22000 min: 0.0
ft_v5_base_cer_28000 min: 0.0
ft_v6_ft_cer_25000 min: 0.0
ft_v7_scratch_cer_23000 min: 0.0
ft_v8_scratch_cer_10000 min: 0.0
ft_v8_scratch_cer_55000 min: 0.0
ft_v8_scratch_cer_59000 min: 0.0
whisper-small-latin-added-tibetan-checkpoint-4000_cer min: 0.0
whisper-small-tibetan-wylie-checkpoint-4000_to_tibetan_cer min: 0.0344827586206896


In [29]:
print("base_lev_dist min:" ,df['base_lev_dist'].min())
print("ft_v1_lev_dist_5000 min:" ,df['ft_v1_lev_dist_5000'].min())
print("ft_v1_lev_dist_10000 min:" ,df['ft_v1_lev_dist_10000'].min())
print("ft_v1_lev_dist_19000 min:" ,df['ft_v1_lev_dist_19000'].min())
print("ft_v2_lev_dist_20000 min:" ,df['ft_v2_lev_dist_20000'].min())
print("ft_v2_lev_dist_32000 min:" ,df['ft_v2_lev_dist_32000'].min())
print("ft_v2_lev_dist_43000 min:" ,df['ft_v2_lev_dist_43000'].min())
print("ft_v3_lev_dist_1000 min:" ,df['ft_v3_lev_dist_1000'].min())
print("ft_v3_lev_dist_25000 min:" ,df['ft_v3_lev_dist_25000'].min())
print("ft_v4_lev_dist_22000 min:" ,df['ft_v4_lev_dist_22000'].min())
print("ft_v5_base_lev_dist_28000 min:" ,df['ft_v5_base_lev_dist_28000'].min())
print("ft_v6_ft_lev_dist_25000 min:" ,df['ft_v6_ft_lev_dist_25000'].min())
print("ft_v7_scratch_lev_dist_23000 min:" ,df['ft_v7_scratch_lev_dist_23000'].min())
print("ft_v8_scratch_lev_dist_10000 min:" ,df['ft_v8_scratch_lev_dist_10000'].min())
print("ft_v8_scratch_lev_dist_55000 min:" ,df['ft_v8_scratch_lev_dist_55000'].min())
print("ft_v8_scratch_lev_dist_59000 min:" ,df['ft_v8_scratch_lev_dist_59000'].min())
print("whisper-small-latin-added-tibetan-checkpoint-4000_lev_dist min:" ,df['whisper-small-latin-added-tibetan-checkpoint-4000_lev_dist'].min())
print("whisper-small-tibetan-wylie-checkpoint-4000_to_tibetan_lev_dist min:" ,df['whisper-small-tibetan-wylie-checkpoint-4000_to_tibetan_lev_dist'].min())

base_lev_dist min: 0
ft_v1_lev_dist_5000 min: 0
ft_v1_lev_dist_10000 min: 0
ft_v1_lev_dist_19000 min: 0
ft_v2_lev_dist_20000 min: 0
ft_v2_lev_dist_32000 min: 0
ft_v2_lev_dist_43000 min: 0
ft_v3_lev_dist_1000 min: 0
ft_v3_lev_dist_25000 min: 0
ft_v4_lev_dist_22000 min: 0
ft_v5_base_lev_dist_28000 min: 0
ft_v6_ft_lev_dist_25000 min: 0
ft_v7_scratch_lev_dist_23000 min: 0
ft_v8_scratch_lev_dist_10000 min: 0
ft_v8_scratch_lev_dist_55000 min: 0
ft_v8_scratch_lev_dist_59000 min: 0
whisper-small-latin-added-tibetan-checkpoint-4000_lev_dist min: 0
whisper-small-tibetan-wylie-checkpoint-4000_to_tibetan_lev_dist min: 1


In [30]:
print("base_cer max:" ,df['base_cer'].max())
print("ft_v1_cer_5000 max:" ,df['ft_v1_cer_5000'].max())
print("ft_v1_cer_10000 max:" ,df['ft_v1_cer_10000'].max())  
print("ft_v1_cer_19000 max:" ,df['ft_v1_cer_19000'].max())
print("ft_v2_cer_20000 max:" ,df['ft_v2_cer_20000'].max())
print("ft_v2_cer_32000 max:" ,df['ft_v2_cer_32000'].max())
print("ft_v2_cer_43000 max:" ,df['ft_v2_cer_43000'].max())
print("ft_v3_cer_1000 max:" ,df['ft_v3_cer_1000'].max())
print("ft_v3_cer_25000 max:" ,df['ft_v3_cer_25000'].max())
print("ft_v4_cer_22000 max:" ,df['ft_v4_cer_22000'].max())
print("ft_v5_base_cer_28000 max:" ,df['ft_v5_base_cer_28000'].max())
print("ft_v6_ft_cer_25000 max:" ,df['ft_v6_ft_cer_25000'].max())
print("ft_v7_scratch_cer_23000 max:" ,df['ft_v7_scratch_cer_23000'].max())
print("ft_v8_scratch_cer_10000 max:" ,df['ft_v8_scratch_cer_10000'].max())
print("ft_v8_scratch_cer_55000 max:" ,df['ft_v8_scratch_cer_55000'].max())
print("ft_v8_scratch_cer_59000 max:" ,df['ft_v8_scratch_cer_59000'].max())
print("whisper-small-latin-added-tibetan-checkpoint-4000_cer max:" ,df['whisper-small-latin-added-tibetan-checkpoint-4000_cer'].max())
print("whisper-small-tibetan-wylie-checkpoint-4000_to_tibetan_cer max:" ,df['whisper-small-tibetan-wylie-checkpoint-4000_to_tibetan_cer'].max())




base_cer max: 1.181818181818182
ft_v1_cer_5000 max: 1.6666666666666667
ft_v1_cer_10000 max: 1.0909090909090908
ft_v1_cer_19000 max: 1.0
ft_v2_cer_20000 max: 1.0
ft_v2_cer_32000 max: 1.0
ft_v2_cer_43000 max: 1.0
ft_v3_cer_1000 max: 1.0
ft_v3_cer_25000 max: 1.0
ft_v4_cer_22000 max: 1.0
ft_v5_base_cer_28000 max: 1.0
ft_v6_ft_cer_25000 max: 1.0
ft_v7_scratch_cer_23000 max: 1.0
ft_v8_scratch_cer_10000 max: 0.9090909090909092
ft_v8_scratch_cer_55000 max: 1.0
ft_v8_scratch_cer_59000 max: 1.0
whisper-small-latin-added-tibetan-checkpoint-4000_cer max: 1.0
whisper-small-tibetan-wylie-checkpoint-4000_to_tibetan_cer max: 1.0


In [31]:
print("base_lev_dist max:" ,df['base_lev_dist'].max())
print("ft_v1_lev_dist_5000 max:" ,df['ft_v1_lev_dist_5000'].max())
print("ft_v1_lev_dist_10000 max:" ,df['ft_v1_lev_dist_10000'].max())
print("ft_v1_lev_dist_19000 max:" ,df['ft_v1_lev_dist_19000'].max())
print("ft_v2_lev_dist_20000 max:" ,df['ft_v2_lev_dist_20000'].max())
print("ft_v2_lev_dist_32000 max:" ,df['ft_v2_lev_dist_32000'].max())
print("ft_v2_lev_dist_43000 max:" ,df['ft_v2_lev_dist_43000'].max())
print("ft_v3_lev_dist_1000 max:" ,df['ft_v3_lev_dist_1000'].max())
print("ft_v3_lev_dist_25000 max:" ,df['ft_v3_lev_dist_25000'].max())
print("ft_v4_lev_dist_22000 max:" ,df['ft_v4_lev_dist_22000'].max())
print("ft_v5_base_lev_dist_28000 max:" ,df['ft_v5_base_lev_dist_28000'].max())
print("ft_v6_ft_lev_dist_25000 max:" ,df['ft_v6_ft_lev_dist_25000'].max())
print("ft_v7_scratch_lev_dist_23000 max:" ,df['ft_v7_scratch_lev_dist_23000'].max())
print("ft_v8_scratch_lev_dist_10000 max:" ,df['ft_v8_scratch_lev_dist_10000'].max())
print("ft_v8_scratch_lev_dist_55000 max:" ,df['ft_v8_scratch_lev_dist_55000'].max())
print("ft_v8_scratch_lev_dist_59000 max:" ,df['ft_v8_scratch_lev_dist_59000'].max())
print("whisper-small-latin-added-tibetan-checkpoint-4000_lev_dist max:" ,df['whisper-small-latin-added-tibetan-checkpoint-4000_lev_dist'].max())
print("whisper-small-tibetan-wylie-checkpoint-4000_to_tibetan_lev_dist max:" ,df['whisper-small-tibetan-wylie-checkpoint-4000_to_tibetan_lev_dist'].max())



base_lev_dist max: 134
ft_v1_lev_dist_5000 max: 155
ft_v1_lev_dist_10000 max: 154
ft_v1_lev_dist_19000 max: 141
ft_v2_lev_dist_20000 max: 130
ft_v2_lev_dist_32000 max: 129
ft_v2_lev_dist_43000 max: 124
ft_v3_lev_dist_1000 max: 143
ft_v3_lev_dist_25000 max: 128
ft_v4_lev_dist_22000 max: 134
ft_v5_base_lev_dist_28000 max: 149
ft_v6_ft_lev_dist_25000 max: 135
ft_v7_scratch_lev_dist_23000 max: 125
ft_v8_scratch_lev_dist_10000 max: 205
ft_v8_scratch_lev_dist_55000 max: 173
ft_v8_scratch_lev_dist_59000 max: 185
whisper-small-latin-added-tibetan-checkpoint-4000_lev_dist max: 379
whisper-small-tibetan-wylie-checkpoint-4000_to_tibetan_lev_dist max: 526


In [ ]:
!pip install matplotlib seaborn 

In [33]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns

# Let's analyze the CER columns from the provided data
# First, I'll create a pandas DataFrame with the data

# These are the column names for CER values
cer_columns = [
    'base_cer', 
    'ft_v1_cer_5000', 'ft_v1_cer_10000', 'ft_v1_cer_19000',
    'ft_v2_cer_20000', 'ft_v2_cer_32000', 'ft_v2_cer_43000',
    'ft_v3_cer_1000', 'ft_v3_cer_25000',
    'ft_v4_cer_22000', 'ft_v5_base_cer_28000',
    'ft_v6_ft_cer_25000', 'ft_v7_scratch_cer_23000',
    'ft_v8_scratch_cer_10000', 'ft_v8_scratch_cer_55000', 'ft_v8_scratch_cer_59000',
    'whisper-small-latin-added-tibetan-checkpoint-4000_cer',
    'whisper-small-tibetan-wylie-checkpoint-4000_to_tibetan_cer'
]

# Let's calculate mean and std for the CER columns
df = pd.read_csv('processed_test_with_cer_and_lev_dist.csv')

# Calculate and display mean and std for each CER column
cer_stats = pd.DataFrame({
    'mean': df[cer_columns].mean(),
    'std': df[cer_columns].std()
})

print(cer_stats)

                                                        mean       std
base_cer                                            0.276695  0.160751
ft_v1_cer_5000                                      0.274089  0.181720
ft_v1_cer_10000                                     0.233720  0.172448
ft_v1_cer_19000                                     0.229333  0.171905
ft_v2_cer_20000                                     0.222694  0.174550
ft_v2_cer_32000                                     0.217564  0.167088
ft_v2_cer_43000                                     0.215314  0.168142
ft_v3_cer_1000                                      0.233648  0.175689
ft_v3_cer_25000                                     0.222136  0.171305
ft_v4_cer_22000                                     0.226609  0.171463
ft_v5_base_cer_28000                                0.215194  0.160661
ft_v6_ft_cer_25000                                  0.220910  0.169975
ft_v7_scratch_cer_23000                             0.218550  0.167179
ft_v8_

In [35]:
import pandas as pd
import numpy as np


# Let's analyze the CER columns from the provided data
# First, I'll create a pandas DataFrame with the data

# These are the column names for CER values
cer_columns = [
    'base_lev_dist', 
    'ft_v1_lev_dist_5000', 'ft_v1_lev_dist_10000', 'ft_v1_lev_dist_19000',
    'ft_v2_lev_dist_20000', 'ft_v2_lev_dist_32000', 'ft_v2_lev_dist_43000',
    'ft_v3_lev_dist_1000', 'ft_v3_lev_dist_25000',
    'ft_v4_lev_dist_22000', 'ft_v5_base_lev_dist_28000',
    'ft_v6_ft_lev_dist_25000', 'ft_v7_scratch_lev_dist_23000',
    'ft_v8_scratch_lev_dist_10000', 'ft_v8_scratch_lev_dist_55000', 'ft_v8_scratch_lev_dist_59000',
    'whisper-small-latin-added-tibetan-checkpoint-4000_lev_dist',
    'whisper-small-tibetan-wylie-checkpoint-4000_to_tibetan_lev_dist'
]

# Let's calculate mean and std for the CER columns
df = pd.read_csv('processed_test_with_cer_and_lev_dist.csv')

# Calculate and display mean and std for each CER column
cer_stats = pd.DataFrame({
    'mean': df[cer_columns].mean(),
    'std': df[cer_columns].std()
})

print(cer_stats)

                                                         mean        std
base_lev_dist                                       19.172452  21.793657
ft_v1_lev_dist_5000                                 18.441209  19.909328
ft_v1_lev_dist_10000                                15.658455  17.421535
ft_v1_lev_dist_19000                                15.333707  16.935004
ft_v2_lev_dist_20000                                14.883539  16.348705
ft_v2_lev_dist_32000                                14.730123  16.143831
ft_v2_lev_dist_43000                                14.608063  16.028661
ft_v3_lev_dist_1000                                 15.440090  16.876830
ft_v3_lev_dist_25000                                14.991041  16.404976
ft_v4_lev_dist_22000                                15.169093  16.709717
ft_v5_base_lev_dist_28000                           14.969765  16.644940
ft_v6_ft_lev_dist_25000                             15.213886  17.197812
ft_v7_scratch_lev_dist_23000                       

In [ ]:
#!/usr/bin/env python3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def analyze_cer():
    # Read the CSV file
    print("Reading CSV file...")
    df = pd.read_csv('/home/gangagyatso/Downloads/processed_test_with_cer_and_lev_dist (1).csv')

    # Define CER columns
    cer_columns = [
        'base_cer', 
        'ft_v1_cer_5000', 'ft_v1_cer_10000', 'ft_v1_cer_19000',
        'ft_v2_cer_20000', 'ft_v2_cer_32000', 'ft_v2_cer_43000',
        'ft_v3_cer_1000', 'ft_v3_cer_25000',
        'ft_v4_cer_22000', 'ft_v5_base_cer_28000',
        'ft_v6_ft_cer_25000', 'ft_v7_scratch_cer_23000'
    ]

    # Calculate mean and std for each CER column
    means = df[cer_columns].mean()
    stds = df[cer_columns].std()

    # Print the statistics
    print('\nCER Statistics:')
    stats_df = pd.DataFrame({'Mean': means, 'Std Dev': stds})
    print(stats_df)

    # Create a bar chart with error bars
    plt.figure(figsize=(14, 10))
    x = np.arange(len(means))
    bars = plt.bar(x, means, yerr=stds, align='center', alpha=0.7, capsize=10)
    
    # Add value labels on top of the bars
    for i, bar in enumerate(bars):
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                 f'{means.iloc[i]:.3f}', ha='center', va='bottom', 
                 fontweight='bold', fontsize=9)
    
    plt.xticks(x, [col.replace('ft_', '').replace('_cer', '') for col in means.index], 
              rotation=45, ha='right')
    plt.ylabel('Character Error Rate (CER)')
    plt.ylim(0, max(means) + max(stds) + 0.1)  # Add padding at the top
    plt.title('Mean CER with Standard Deviation Error Bars')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()

    # Save the plot
    plt.savefig('cer_comparison.png')
    print('Bar chart saved as cer_comparison.png')

    # Group by model version and create another visualization
    model_groups = {
        'base': ['base_cer'],
        'v1': ['ft_v1_cer_5000', 'ft_v1_cer_10000', 'ft_v1_cer_19000'],
        'v2': ['ft_v2_cer_20000', 'ft_v2_cer_32000', 'ft_v2_cer_43000'],
        'v3': ['ft_v3_cer_1000', 'ft_v3_cer_25000'],
        'v4': ['ft_v4_cer_22000'],
        'v5': ['ft_v5_base_cer_28000'],
        'v6': ['ft_v6_ft_cer_25000'],
        'v7': ['ft_v7_scratch_cer_23000']
    }

    # Calculate grouped means and stds
    group_means = {}
    group_stds = {}

    for group, columns in model_groups.items():
        group_data = df[columns].values.flatten()
        group_means[group] = group_data.mean()
        group_stds[group] = group_data.std()

    # Create grouped bar chart
    plt.figure(figsize=(10, 10))
    x = np.arange(len(group_means))
    bars = plt.bar(x, list(group_means.values()), yerr=list(group_stds.values()), 
                  align='center', alpha=0.7, capsize=10, 
                  color=sns.color_palette("muted", len(group_means)))
    
    # Add value labels on top of the bars
    for i, bar in enumerate(bars):
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                 f'{list(group_means.values())[i]:.3f}', ha='center', va='bottom', 
                 fontweight='bold')
    
    plt.xticks(x, list(group_means.keys()))
    plt.ylabel('Character Error Rate (CER)')
    plt.title('Mean CER by Model Version with Standard Deviation')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()

    # Save the grouped plot
    plt.savefig('cer_by_model_version.png')
    print('Bar chart by model version saved as cer_by_model_version.png')
    
   # Add histogram for distribution analysis
    plt.figure(figsize=(16, 14))  # Increased figure size to accommodate more subplots

    # Create subplots for each model version
    for i, (group, columns) in enumerate(model_groups.items(), 1):
        plt.subplot(3, 3, i)  # Changed from 3x2 to 3x3
        for col in columns:
            sns.histplot(df[col].dropna(), label=col.replace('ft_', '').replace('_cer', ''), kde=True, alpha=0.6)
        plt.title(f'CER Distribution - {group}')
        plt.xlabel('CER Value')
        plt.ylabel('Count')
        plt.legend()
    
    plt.tight_layout()
    plt.savefig('cer_distributions.png')
    print('CER distributions saved as cer_distributions.png')

    # Return the statistics for further analysis
    return stats_df

if __name__ == "__main__":
    analyze_cer()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

def analyze_cer():
    # Read the CSV file
    print("Reading CSV file...")
    df = pd.read_csv('processed_test_with_cer_and_lev_dist.csv')

    # Define CER columns
    cer_columns = [
        'base_cer', 
        'ft_v1_cer_5000', 'ft_v1_cer_10000', 'ft_v1_cer_19000',
        'ft_v2_cer_20000', 'ft_v2_cer_32000', 'ft_v2_cer_43000',
        'ft_v3_cer_1000', 'ft_v3_cer_25000',
        'ft_v4_cer_22000',
        'ft_v5_base_cer_28000',
        'ft_v6_ft_cer_25000',
        'ft_v7_scratch_cer_23000'
    ]

    # Calculate mean and std for each CER column
    means = df[cer_columns].mean()
    stds = df[cer_columns].std()

    # Print the statistics
    print('\nCER Statistics:')
    stats_df = pd.DataFrame({'Mean': means, 'Std Dev': stds})
    print(stats_df)

    # Create a bar chart with error bars
    plt.figure(figsize=(14, 7))
    x = np.arange(len(means))
    bars = plt.bar(x, means, yerr=stds, align='center', alpha=0.7, capsize=10)
    
    # Add value labels on top of the bars
    for i, bar in enumerate(bars):
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                 f'{means.iloc[i]:.3f}', ha='center', va='bottom', 
                 fontweight='bold', fontsize=9)
    
    plt.xticks(x, [col.replace('ft_', '').replace('_cer', '') for col in means.index], 
              rotation=45, ha='right')
    plt.ylabel('Character Error Rate (CER)')
    plt.ylim(0, max(means) + max(stds) + 0.1)  # Add padding at the top
    plt.title('Mean CER with Standard Deviation Error Bars')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()

    # Save the plot
    plt.savefig('cer_comparison.png')
    print('Bar chart saved as cer_comparison.png')

    # Group by model version and create another visualization
    model_groups = {
        'base': ['base_cer'],
        'v1': ['ft_v1_cer_5000', 'ft_v1_cer_10000', 'ft_v1_cer_19000'],
        'v2': ['ft_v2_cer_20000', 'ft_v2_cer_32000', 'ft_v2_cer_43000'],
        'v3': ['ft_v3_cer_1000', 'ft_v3_cer_25000'],
        'v4': ['ft_v4_cer_22000'],
        'v5': ['ft_v5_base_cer_28000'],
        'v6': ['ft_v6_ft_cer_25000'],
        'v7': ['ft_v7_scratch_cer_23000']
    }

    # Calculate grouped means and stds
    group_means = {}
    group_stds = {}

    for group, columns in model_groups.items():
        group_data = df[columns].values.flatten()
        group_means[group] = group_data.mean()
        group_stds[group] = group_data.std()

    # Create grouped bar chart
    plt.figure(figsize=(10, 6))
    x = np.arange(len(group_means))
    bars = plt.bar(x, list(group_means.values()), yerr=list(group_stds.values()), 
                  align='center', alpha=0.7, capsize=10, 
                  color=sns.color_palette("muted", len(group_means)))
    
    # Add value labels on top of the bars
    for i, bar in enumerate(bars):
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                 f'{list(group_means.values())[i]:.3f}', ha='center', va='bottom', 
                 fontweight='bold')
    
    plt.xticks(x, list(group_means.keys()))
    plt.ylabel('Character Error Rate (CER)')
    plt.title('Mean CER by Model Version with Standard Deviation')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()

    # Save the grouped plot
    plt.savefig('cer_by_model_version.png')
    print('Bar chart by model version saved as cer_by_model_version.png')
    
    # Add histogram for distribution analysis
    plt.figure(figsize=(16, 14))  # Increased figure size for more subplots
    
    # Create subplots for each model version
    for i, (group, columns) in enumerate(model_groups.items(), 1):
        plt.subplot(3, 3, i)  # Changed from 3x2 to 3x3
        for col in columns:
            sns.histplot(df[col].dropna(), label=col.replace('ft_', '').replace('_cer', ''), kde=True, alpha=0.6)
        plt.title(f'CER Distribution - {group}')
        plt.xlabel('CER Value')
        plt.ylabel('Count')
        plt.legend()
    
    plt.tight_layout()
    plt.savefig('cer_distributions.png')
    print('CER distributions saved as cer_distributions.png')
    
    # Calculate percentage improvement over base model
    base_cer = means['base_cer']
    improved_columns = [col for col in cer_columns if col != 'base_cer']
    
    improvement_percentages = {}
    for col in improved_columns:
        improvement = (base_cer - means[col]) / base_cer * 100
        improvement_percentages[col] = improvement
    
    # Create a sorted list of columns by model version and checkpoint
    # Fixed: Use regex to extract model version and checkpoint number
    def extract_version_and_checkpoint(col_name):
        # Extract version number (v1, v2, etc)
        version_match = re.search(r'v(\d+)', col_name)
        version = int(version_match.group(1)) if version_match else 0
        
        # Extract checkpoint number (the last number in the string)
        checkpoint_match = re.search(r'_(\d+)$', col_name)
        checkpoint = int(checkpoint_match.group(1)) if checkpoint_match else 0
        
        return (version, checkpoint)
    
    sorted_cols = sorted(improved_columns, key=extract_version_and_checkpoint)
    
    # Create improvement visualization
    plt.figure(figsize=(14, 7))
    
    x = np.arange(len(sorted_cols))
    improvements = [improvement_percentages[col] for col in sorted_cols]
    
    # Define color mapping for different model versions
    version_colors = {
        'v1': '#3498db',  # blue
        'v2': '#2ecc71',  # green
        'v3': '#e74c3c',  # red
        'v4': '#9b59b6',  # purple
        'v5': '#34495e',  # dark blue
        'v6': '#f1c40f',  # yellow
        'v7': '#e67e22'   # orange
    }
    
    # Map colors to bars based on model version
    colors = [version_colors[re.search(r'v\d+', col).group(0)] for col in sorted_cols]
    
    bars = plt.bar(x, improvements, align='center', alpha=0.8, color=colors)
    
    # Add value labels on top of the bars
    for i, bar in enumerate(bars):
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.5,
                f'{improvements[i]:.2f}%', ha='center', va='bottom',
                fontweight='bold', fontsize=10)
    
    # Format x-axis labels to be more readable
    labels = []
    for col in sorted_cols:
        version = re.search(r'v\d+', col).group(0)
        checkpoint = re.search(r'_(\d+)$', col).group(1)
        labels.append(f"{version} ({checkpoint})")
    
    plt.xticks(x, labels, rotation=45, ha='right')
    plt.ylabel('Percentage Improvement Over Base Model (%)')
    plt.title('CER Improvement Percentage Over Base Model')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Add a horizontal line at zero to emphasize the improvement
    plt.axhline(y=0, color='black', linestyle='-', alpha=0.3)
    
    # Create a legend for the model versions
    from matplotlib.patches import Patch
    legend_elements = [Patch(facecolor=color, label=version) 
                      for version, color in version_colors.items()]
    plt.legend(handles=legend_elements, title="Model Version")
    
    plt.tight_layout()
    plt.savefig('cer_improvement_percentage.png', dpi=300, bbox_inches='tight')
    print('CER improvement percentage saved as cer_improvement_percentage.png')
    
    # Create a table with all the important metrics
    improvement_df = pd.DataFrame({
        'Model': [col.replace('ft_', '').replace('_cer', '') for col in improved_columns],
        'CER': [means[col] for col in improved_columns],
        'Improvement (%)': [improvement_percentages[col] for col in improved_columns]
    })
    
    improvement_df = improvement_df.sort_values('Improvement (%)', ascending=False)
    print("\nCER Improvement Rankings:")
    print(improvement_df)
    
    # Return the statistics for further analysis
    return stats_df

if __name__ == "__main__":
    analyze_cer()

In [ ]:
import pandas as pd
import numpy as np
import os

def create_cer_bins():
    """
    Creates a CSV file with files binned by CER ranges for each model.
    
    Bins are: 0-0.05, 0.05-0.1, 0.1-0.15, 0.15-0.2, 0.2-0.25, 0.25-0.3, 0.3-0.35,
    0.35-0.4, 0.4-0.45, 0.45-0.5, 0.5+
    """
    print("Reading CSV file...")
    df = pd.read_csv('processed_test_with_cer_and_lev_dist.csv')

    # Define CER columns
    cer_columns = [
        'base_cer', 
        'ft_v1_cer_5000', 'ft_v1_cer_10000', 'ft_v1_cer_19000',
        'ft_v2_cer_20000', 'ft_v2_cer_32000', 'ft_v2_cer_43000',
        'ft_v3_cer_1000', 'ft_v3_cer_25000',
        'ft_v4_cer_22000',
        'ft_v5_base_cer_28000',
        'ft_v6_ft_cer_25000',
        'ft_v7_scratch_cer_23000'
    ]
    
    # Define the bins
    bin_edges = [0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 1.0]
    bin_labels = [
        '0.00-0.05', '0.05-0.10', '0.10-0.15', '0.15-0.20', '0.20-0.25', 
        '0.25-0.30', '0.30-0.35', '0.35-0.40', '0.40-0.45', '0.45-0.50', '0.50+'
    ]
    
    # Create a dictionary to store binned file names for each CER column
    binned_data = {}
    
    # For each CER column, categorize files into bins
    for col in cer_columns:
        # Initialize lists for each bin
        binned_data[col] = {bin_label: [] for bin_label in bin_labels}
        
        # Categorize each file based on its CER value
        for _, row in df.iterrows():
            cer_value = row[col]
            
            # Skip if CER value is missing
            if pd.isna(cer_value):
                continue
                
            # Find the appropriate bin
            bin_idx = np.digitize([cer_value], bin_edges)[0] - 1
            
            # Handle edge case where CER value is exactly 1.0
            if bin_idx >= len(bin_labels):
                bin_idx = len(bin_labels) - 1
                
            bin_label = bin_labels[bin_idx]
            
            # Add file name to the bin
            binned_data[col][bin_label].append(row['file_name'])
    
    # Create detailed bin CSV - shows files in each bin for each model
    detailed_rows = []
    
    for col in cer_columns:
        model_name = col.replace('_cer', '')
        for bin_label in bin_labels:
            files_in_bin = binned_data[col][bin_label]
            count = len(files_in_bin)
            
            # Only add to the CSV if there are files in this bin
            if count > 0:
                file_list = ';'.join(files_in_bin)
                detailed_rows.append({
                    'Model': model_name,
                    'CER_Range': bin_label,
                    'Count': count,
                    'File_Names': file_list
                })
    
    # Create DataFrame from the rows
    detailed_df = pd.DataFrame(detailed_rows)
    detailed_csv_path = 'cer_bins_detailed.csv'
    detailed_df.to_csv(detailed_csv_path, index=False)
    print(f"Detailed CER bins saved as {detailed_csv_path}")
    
    # Create summary CSV - shows count of files in each bin for each model
    summary_data = {}
    
    # Initialize the dictionary with model names as keys
    for col in cer_columns:
        model_name = col.replace('_cer', '')
        summary_data[model_name] = {}
        
        # Count files in each bin
        for bin_label in bin_labels:
            summary_data[model_name][bin_label] = len(binned_data[col][bin_label])
    
    # Convert to DataFrame
    summary_df = pd.DataFrame.from_dict(summary_data, orient='index')
    summary_csv_path = 'cer_bins_summary.csv'
    summary_df.to_csv(summary_csv_path)
    print(f"Summary CER bins saved as {summary_csv_path}")
    
    # Create distribution comparison CSV - shows percentage of files in each bin
    distribution_data = {}
    
    # Calculate percentages for each model
    for col in cer_columns:
        model_name = col.replace('_cer', '')
        distribution_data[model_name] = {}
        
        # Get total number of files with valid CER values for this model
        total_files = sum(len(binned_data[col][bin_label]) for bin_label in bin_labels)
        
        # Calculate percentage for each bin
        for bin_label in bin_labels:
            count = len(binned_data[col][bin_label])
            percentage = (count / total_files * 100) if total_files > 0 else 0
            distribution_data[model_name][bin_label] = round(percentage, 2)
    
    # Convert to DataFrame
    distribution_df = pd.DataFrame.from_dict(distribution_data, orient='index')
    distribution_csv_path = 'cer_bins_distribution.csv'
    distribution_df.to_csv(distribution_csv_path)
    print(f"Distribution CER bins saved as {distribution_csv_path}")
    
    return detailed_df, summary_df, distribution_df

if __name__ == "__main__":
    create_cer_bins()


In [ ]:
! pip install seaborn

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the summary CSV
df = pd.read_csv('cer_bins_summary.csv', index_col=0)

# Set up plot style
plt.figure(figsize=(16, 7))
sns.set(style="whitegrid")

# Transpose for grouped bar chart (bins on x-axis, models as bars)
df_T = df.transpose()

# Plot
ax = df_T.plot(kind='bar', figsize=(18, 8), width=0.85)
plt.title('Count of Files in Each CER Bin for Each Model', fontsize=18)
plt.xlabel('CER Bin', fontsize=14)
plt.ylabel('Number of Files', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.legend(title='Model', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
! pip install tibetan_wer

In [38]:
#!/usr/bin/env python3
import pandas as pd
import numpy as np

# Import the Tibetan WER and SER metrics
from tibetan_wer.metrics import wer, ser

# Load the CSV file
print("Loading CSV file...")
csv_file = "processed_test_with_cer_and_lev_dist.csv"  # Using the file with CER already calculated
df = pd.read_csv(csv_file)

# Define the inference columns and their corresponding WER/SER column names
inference_columns = [
    "whisper-small-latin-added-tibetan-checkpoint-4000",
    "whisper-small-tibetan-wylie-checkpoint-4000_to_tibetan"
]


Loading CSV file...


In [39]:
# Calculate WER and SER for each inference column against the ground truth "uni" column
print("Calculating WER and SER metrics...")

# Initialize lists to store results
results = []

# Loop through each inference column
for col in inference_columns:
    print(f"Processing {col}...")
    
    # Get predictions and references
    predictions = df[col].tolist()
    references = df["uni"].tolist()
    
    # Calculate WER
    wer_result = wer(predictions, references)
    
    # Calculate SER
    ser_result = ser(predictions, references)
    
    # Store results
    results.append({
        "model": col,
        "micro_wer": wer_result["micro_wer"],
        "macro_wer": wer_result["macro_wer"],
        "micro_ser": ser_result["micro_ser"],
        "macro_ser": ser_result["macro_ser"],
        "substitutions": ser_result["substitutions"],
        "insertions": ser_result["insertions"],
        "deletions": ser_result["deletions"]
    })

# Convert results to DataFrame for easier comparison
results_df = pd.DataFrame(results)
print("\nResults:")
print(results_df)

# Save results to CSV
results_df.to_csv("wer_ser_results.csv", index=False)
print("\nResults saved to wer_ser_results.csv")

Calculating WER and SER metrics...
Processing whisper-small-latin-added-tibetan-checkpoint-4000...
Processing whisper-small-tibetan-wylie-checkpoint-4000_to_tibetan...

Results:
                                               model  micro_wer  macro_wer  \
0  whisper-small-latin-added-tibetan-checkpoint-4000   0.607723   0.587186   
1  whisper-small-tibetan-wylie-checkpoint-4000_to...   0.675397   0.712424   

   micro_ser  macro_ser  substitutions  insertions  deletions  
0   0.565648    0.56568           7289         543       1478  
1   0.616562    0.65628           6741        1561       1846  

Results saved to wer_ser_results.csv


In [ ]:
! pip install tibetan-wer matplotlib seaborn pandas

In [ ]:
#!/usr/bin/env python3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Use the results data directly
data = """model,micro_wer,macro_wer,micro_ser,macro_ser,substitutions,insertions,deletions
base_transcript,0.5535589116166444,0.5972172117029008,0.49286104866638314,0.5178815698935828,7100,504,508
ft_v1_inference_checkpoint-5000,0.5182565475052571,0.5738740985951234,0.48210705389148795,0.5440424343257099,6651,480,804
ft_v1_inference_checkpoint-10000,0.40980054801503857,0.4679552413294852,0.39759402150798956,0.479418050592188,5078,652,814
ft_v1_inference_checkpoint-19000,0.3941884916841904,0.4592440920690446,0.3864754845373352,0.4704815022591255,4962,637,762
ft_v2_inference_checkpoint-20000,0.3919581979226407,0.4510195125772621,0.38428823136278023,0.4665372243930654,4999,665,661
ft_v2_inference_checkpoint-32000,0.38641432485821703,0.4429847994193427,0.3797314539157908,0.4655471996665233,4905,638,707
ft_v2_inference_checkpoint-43000,0.38099789715159627,0.4375001935072245,0.3755392186645604,0.4607023706973961,4858,654,669
ft_v3_inference_checkpoint-1000,0.41247690052889824,0.47391160986923914,0.40148247159608724,0.48557158754267693,5217,654,737
ft_v3_inference_checkpoint-25000,0.3911298030969222,0.4519507580126224,0.3827693055471171,0.46346152957429354,4951,649,700
ft_v4_inference_checkpoint-22000,0.40890843051041864,0.47620171588845034,0.3924296737347348,0.47617469678609764,5121,600,738
ft_v5_base_inference_checkpoint-28000,0.4128592366023068,0.4590233735658298,0.40148247159608724,0.46976534032892286,5254,596,758
ft_v6_ft_inference_checkpoint-25000,0.40400178423500926,0.4508872276075435,0.3927334588978674,0.4641250916524783,5059,647,758
ft_v7_scratch_inference_checkpoint-23000,0.41171222838208116,0.46476922104444907,0.3968041800838447,0.470994750522651,5270,645,616
"""

# Convert the string to a DataFrame
from io import StringIO
results_df = pd.read_csv(StringIO(data))

print("Creating visualizations...")

# Clean up model names for better display
results_df['model_display'] = results_df['model'].apply(
    lambda x: x.replace('_inference_checkpoint-', ' ck-').replace('ft_', 'FT ')
)

# Extract version and checkpoint numbers for sorting
results_df['version'] = results_df['model'].str.extract(r'ft_v(\d+)').astype(float)
results_df.loc[results_df['model'] == 'base_transcript', 'version'] = 0
results_df['checkpoint'] = results_df['model'].str.extract(r'checkpoint-(\d+)').astype(float)
results_df.loc[results_df['model'] == 'base_transcript', 'checkpoint'] = 0

# Sort by version and checkpoint
results_df = results_df.sort_values(['version', 'checkpoint'])

# Reset index for better plotting
results_df = results_df.reset_index(drop=True)

# Set the style
sns.set_style("whitegrid")
plt.rcParams['font.family'] = 'DejaVu Sans'

# 1. Create a comprehensive visualization with multiple subplots
plt.figure(figsize=(16, 12))

# Create subplot for WER
plt.subplot(2, 2, 1)
ax1 = sns.barplot(x='model_display', y='micro_wer', data=results_df, palette='viridis')
plt.title('Micro-Average Word Error Rate (WER)', fontsize=14)
plt.xlabel('')
plt.xticks(rotation=45, ha='right')
plt.ylim(0, min(1.0, results_df['micro_wer'].max() * 1.1))
for i, p in enumerate(ax1.patches):
    ax1.annotate(f'{p.get_height():.3f}', 
                (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha = 'center', va = 'bottom', fontsize=9)

# Create subplot for SER
plt.subplot(2, 2, 2)
ax2 = sns.barplot(x='model_display', y='micro_ser', data=results_df, palette='viridis')
plt.title('Micro-Average Syllable Error Rate (SER)', fontsize=14)
plt.xlabel('')
plt.xticks(rotation=45, ha='right')
plt.ylim(0, min(1.0, results_df['micro_ser'].max() * 1.1))
for i, p in enumerate(ax2.patches):
    ax2.annotate(f'{p.get_height():.3f}', 
                (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha = 'center', va = 'bottom', fontsize=9)

# Create subplot for error breakdown
plt.subplot(2, 2, 3)
error_types = pd.melt(results_df, 
                      id_vars=['model_display'],
                      value_vars=['substitutions', 'insertions', 'deletions'],
                      var_name='Error_Type', 
                      value_name='Count')

# Create a pivot table for the stacked bar chart
error_pivot = error_types.pivot(index='model_display', columns='Error_Type', values='Count')

# Plot stacked bar chart
error_pivot.plot(kind='bar', stacked=True, ax=plt.gca(), colormap='tab10')
plt.title('Error Type Breakdown', fontsize=14)
plt.xlabel('')
plt.xticks(rotation=45, ha='right')
plt.legend(title='')

# Create subplot for version comparison
plt.subplot(2, 2, 4)

# Prepare data for grouped bar plot
metrics_data = pd.melt(results_df, 
                      id_vars=['model_display', 'version'],
                      value_vars=['micro_wer', 'micro_ser'],
                      var_name='Metric', 
                      value_name='Value')
metrics_data['Metric'] = metrics_data['Metric'].apply(lambda x: 'WER' if x == 'micro_wer' else 'SER')

sns.lineplot(x='model_display', y='Value', hue='Metric', data=metrics_data, 
            markers=True, dashes=False, palette=['#1f77b4', '#ff7f0e'])
plt.title('WER and SER Trend Across Models', fontsize=14)
plt.xlabel('')
plt.xticks(rotation=45, ha='right')
plt.ylim(0.35, min(1.0, metrics_data['Value'].max() * 1.05))
plt.legend(title='')
plt.grid(True, linestyle='--', alpha=0.7)

plt.tight_layout()
plt.savefig('wer_ser_comparison_all.png', dpi=300, bbox_inches='tight')
plt.savefig('wer_ser_comparison_all.pdf', bbox_inches='tight')

# 2. Create a focused comparison of WER and SER
plt.figure(figsize=(14, 8))

# Prepare data for comparison
main_metrics = pd.melt(results_df,
                      id_vars=['model_display'],
                      value_vars=['micro_wer', 'micro_ser'],
                      var_name='Metric',
                      value_name='Value')
main_metrics['Metric'] = main_metrics['Metric'].apply(lambda x: 'WER' if x == 'micro_wer' else 'SER')

# Create grouped bar plot
ax = sns.barplot(x='model_display', y='Value', hue='Metric', data=main_metrics, 
               palette=['#3498db', '#e74c3c'])
plt.title('WER and SER Comparison Across Models', fontsize=16)
plt.xlabel('Model', fontsize=14)
plt.ylabel('Error Rate', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.ylim(0.35, min(1.0, main_metrics['Value'].max() * 1.05))
plt.legend(title='', loc='upper right', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Add value labels on top of bars
for i, p in enumerate(plt.gca().patches):
    height = p.get_height()
    if not np.isnan(height):
        plt.gca().text(p.get_x() + p.get_width()/2., height + 0.005,
                      f'{height:.3f}',
                      ha="center", fontsize=10)

plt.tight_layout()
plt.savefig('wer_ser_main_comparison.png', dpi=300, bbox_inches='tight')
plt.savefig('wer_ser_main_comparison.pdf', bbox_inches='tight')

# 3. Create a progression plot showing improvement across checkpoints
plt.figure(figsize=(14, 7))

# Group by version
versions = results_df['version'].unique()
version_colors = plt.cm.tab10(np.linspace(0, 1, len(versions)))

for i, ver in enumerate(versions):
    ver_data = results_df[results_df['version'] == ver]
    if len(ver_data) > 0:
        if ver == 0:  # Base model
            plt.axhline(y=ver_data['micro_wer'].values[0], color='red', linestyle='--', 
                       label=f'Base WER: {ver_data["micro_wer"].values[0]:.3f}')
            plt.axhline(y=ver_data['micro_ser'].values[0], color='blue', linestyle='--', 
                       label=f'Base SER: {ver_data["micro_ser"].values[0]:.3f}')
        else:
            # Plot WER
            plt.plot(ver_data['checkpoint'], ver_data['micro_wer'], 
                   marker='o', markersize=8, linewidth=2, color=version_colors[i], 
                   label=f'v{int(ver)} WER')
            
            # Plot SER with different marker
            plt.plot(ver_data['checkpoint'], ver_data['micro_ser'], 
                   marker='s', markersize=8, linewidth=2, linestyle='--', color=version_colors[i], 
                   label=f'v{int(ver)} SER')
            
            # Add annotations
            for j, row in ver_data.iterrows():
                plt.annotate(f'{row["micro_wer"]:.3f}', 
                           (row['checkpoint'], row['micro_wer']),
                           textcoords="offset points", 
                           xytext=(0,10), 
                           ha='center',
                           fontsize=9)
                plt.annotate(f'{row["micro_ser"]:.3f}', 
                           (row['checkpoint'], row['micro_ser']),
                           textcoords="offset points", 
                           xytext=(0,-15), 
                           ha='center',
                           fontsize=9)

plt.title('WER and SER Progression Across Checkpoints', fontsize=16)
plt.xlabel('Checkpoint', fontsize=14)
plt.ylabel('Error Rate', fontsize=14)
plt.ylim(0.35, 0.6)
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend(fontsize=12)

plt.tight_layout()
plt.savefig('wer_ser_progression.png', dpi=300, bbox_inches='tight')
plt.savefig('wer_ser_progression.pdf', bbox_inches='tight')

# 4. Improvement over baseline
plt.figure(figsize=(14, 7))

# Calculate improvement percentage compared to base model
base_wer = results_df.loc[results_df['model'] == 'base_transcript', 'micro_wer'].values[0]
base_ser = results_df.loc[results_df['model'] == 'base_transcript', 'micro_ser'].values[0]

results_df['wer_improvement'] = (base_wer - results_df['micro_wer']) / base_wer * 100
results_df['ser_improvement'] = (base_ser - results_df['micro_ser']) / base_ser * 100

# Filter out base model for the improvement chart
improvement_df = results_df[results_df['model'] != 'base_transcript'].copy()

# Create a bar chart for improvement percentage
improvement_data = pd.melt(improvement_df,
                         id_vars=['model_display'],
                         value_vars=['wer_improvement', 'ser_improvement'],
                         var_name='Metric',
                         value_name='Improvement_Percentage')
improvement_data['Metric'] = improvement_data['Metric'].apply(
    lambda x: 'WER Improvement' if x == 'wer_improvement' else 'SER Improvement'
)

ax = sns.barplot(x='model_display', y='Improvement_Percentage', hue='Metric', 
               data=improvement_data, palette=['#2ecc71', '#9b59b6'])
plt.title('Percentage Improvement Over Base Model', fontsize=16)
plt.xlabel('Model', fontsize=14)
plt.ylabel('Improvement Percentage (%)', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.legend(title='', fontsize=12)

# Add value labels on top of bars
for i, p in enumerate(plt.gca().patches):
    height = p.get_height()
    if not np.isnan(height):
        plt.gca().text(p.get_x() + p.get_width()/2., height + 0.5,
                      f'{height:.1f}%',
                      ha="center", fontsize=10)

plt.tight_layout()
plt.savefig('improvement_over_base.png', dpi=300, bbox_inches='tight')
plt.savefig('improvement_over_base.pdf', bbox_inches='tight')

print("Visualization complete!")
print("Generated files:")
print("1. wer_ser_comparison_all.png/pdf - Comprehensive visualization with multiple metrics")
print("2. wer_ser_main_comparison.png/pdf - Direct comparison of WER and SER across models")
print("3. wer_ser_progression.png/pdf - Progression chart showing improvement across checkpoints")
print("4. improvement_over_base.png/pdf - Percentage improvement over base model")


In [ ]:
#!/usr/bin/env python3
import pandas as pd
import numpy as np

# Import the Tibetan WER and SER metrics
from tibetan_wer.metrics import wer, ser

# Load the CSV file
print("Loading CSV file...")
csv_file = "processed_test_with_cer.csv"  # Using the file with CER already calculated
df = pd.read_csv(csv_file)

# Define the inference columns and their corresponding WER/SER column names
inference_columns = [
    "ft_v2_inference_checkpoint-20000",
    "ft_v2_inference_checkpoint-32000", 
    "ft_v2_inference_checkpoint-43000",
    "ft_v3_inference_checkpoint-1000",
    "ft_v3_inference_checkpoint-25000",
    "ft_v4_inference_checkpoint-22000",
    "ft_v5_base_inference_checkpoint-28000",
    "ft_v6_ft_inference_checkpoint-25000",
    "ft_v7_scratch_inference_checkpoint-23000"
]

# Create the mappings
wer_columns = {col: col.replace("inference", "wer") for col in inference_columns}
ser_columns = {col: col.replace("inference", "ser") for col in inference_columns}

# Function to calculate WER
def calculate_wer(reference, prediction):
    try:
        if not reference or not prediction or pd.isna(reference) or pd.isna(prediction):
            return np.nan
        result = wer([prediction], [reference])
        return min(result, 1.0)  # Cap at 1.0 like the CER calculation
    except Exception as e:
        print(f"Error calculating WER: {e}")
        return np.nan

# Function to calculate SER (returns micro_ser)
def calculate_ser(reference, prediction):
    try:
        if not reference or not prediction or pd.isna(reference) or pd.isna(prediction):
            return np.nan
        result = ser([prediction], [reference])
        return min(result["micro_ser"], 1.0)  # Return micro_ser and cap at 1.0
    except Exception as e:
        print(f"Error calculating SER: {e}")
        return np.nan

# Calculate and add WER columns
print("Calculating WER metrics...")
for inference_col, wer_col in wer_columns.items():
    print(f"  Processing {wer_col}...")
    df[wer_col] = df.apply(
        lambda row: calculate_wer(row['uni'], row[inference_col]), 
        axis=1
    )

# Calculate and add SER columns
print("Calculating SER metrics...")
for inference_col, ser_col in ser_columns.items():
    print(f"  Processing {ser_col}...")
    df[ser_col] = df.apply(
        lambda row: calculate_ser(row['uni'], row[inference_col]), 
        axis=1
    )

# Save the updated DataFrame back to CSV
output_file = "processed_test_with_cer_wer_ser.csv"
df.to_csv(output_file, index=False)
print(f"WER and SER calculations completed and saved to {output_file}")

# Calculate and display the mean and standard deviation for each metric
print("\nMetric Statistics:")
metric_columns = list(wer_columns.values()) + list(ser_columns.values())
stats_df = pd.DataFrame({
    'Mean': df[metric_columns].mean(),
    'Std Dev': df[metric_columns].std()
})
print(stats_df)

# For a single example, show detailed SER breakdown
print("\nDetailed SER breakdown for the first row:")
try:
    example_ref = df['uni'].iloc[0]
    example_pred = df[inference_columns[0]].iloc[0]
    
    if not pd.isna(example_ref) and not pd.isna(example_pred):
        result = ser([example_pred], [example_ref])
        print(f'Reference: {example_ref}')
        print(f'Prediction: {example_pred}')
        print(f'Micro-Average SER Score: {result["micro_ser"]:.3f}')
        print(f'Macro-Average SER Score: {result["macro_ser"]:.3f}')
        print(f'Substitutions: {result["substitutions"]:.0f}')
        print(f'Insertions: {result["insertions"]:.0f}')
        print(f'Deletions: {result["deletions"]:.0f}')
    else:
        print("First row contains NaN values, cannot calculate detailed SER.")
except Exception as e:
    print(f"Error calculating detailed SER: {e}")


In [ ]:
pip install pandas matplotlib seaborn

In [ ]:
df = pd.read_csv("processed_test.csv")
df2 = pd.read_csv("transcription_data_latest.csv")

In [ ]:
df.head()

In [ ]:
df2.head()

In [ ]:
import pandas as pd

# Read both dataframes


# Create a mapping dictionary from df2
transcript_map = dict(zip(df2['file_name'], df2['transcript']))

# Create new 'uni' column in df1 by mapping values from df2
df['uni'] = df['file_name'].map(transcript_map)

# Save the updated df1
df.to_csv('processed_test1_with_uni.csv', index=False)

print("Number of rows in df1:", len(df))
print("Number of rows in df2:", len(df2))
print("Number of matched transcripts:", df['uni'].notna().sum())
print("\nFirst few rows of updated df1:")
print(df[['file_name', 'uni']].head())

In [ ]:
df = pd.read_csv('processed_test1_with_uni.csv')

In [ ]:
len(df)

In [ ]:
df = df.dropna(subset=['uni'])


In [ ]:
len(df)

In [ ]:
df.to_csv('./processed_test1_with_uni_without_na.csv', index=False)

In [ ]:
df.to_csv('./processed_test1_with_uni_without_na.csv', index=False)

In [10]:
import pandas as pd

df = pd.read_csv("/home/gangagyatso/Downloads/processed_test_with_cer_and_lev_dist (1).csv")

In [11]:
df.columns

Index(['file_name', 'path', 'url', 'audio_duration', 'original_id', 'strata',
       'age_group', 'duration_category', 'content_type', 'uni',
       'base_transcript', 'base_cer', 'base_lev_dist',
       'ft_v1_inference_checkpoint-5000', 'ft_v1_cer_5000',
       'ft_v1_lev_dist_5000', 'ft_v1_inference_checkpoint-10000',
       'ft_v1_cer_10000', 'ft_v1_lev_dist_10000',
       'ft_v1_inference_checkpoint-19000', 'ft_v1_cer_19000',
       'ft_v1_lev_dist_19000', 'ft_v2_inference_checkpoint-20000',
       'ft_v2_cer_20000', 'ft_v2_lev_dist_20000',
       'ft_v2_inference_checkpoint-32000', 'ft_v2_cer_32000',
       'ft_v2_lev_dist_32000', 'ft_v2_inference_checkpoint-43000',
       'ft_v2_cer_43000', 'ft_v2_lev_dist_43000',
       'ft_v3_inference_checkpoint-1000', 'ft_v3_cer_1000',
       'ft_v3_lev_dist_1000', 'ft_v3_inference_checkpoint-25000',
       'ft_v3_cer_25000', 'ft_v3_lev_dist_25000',
       'ft_v4_inference_checkpoint-22000', 'ft_v4_cer_22000',
       'ft_v4_lev_dist_22000', 

In [12]:
df.head(2)

,file_name,path,url,audio_duration,original_id,strata,age_group,duration_category,content_type,uni,...,ft_v7_scratch_lev_dist_23000,ft_v8_scartch_inference_checkpoint-10000,ft_v8_scartch_inference_checkpoint-55000,ft_v8_scartch_inference_checkpoint-59000,ft_v8_scratch_cer_10000,ft_v8_scratch_cer_55000,ft_v8_scratch_cer_59000,ft_v8_scratch_lev_dist_10000,ft_v8_scratch_lev_dist_55000,ft_v8_scratch_lev_dist_59000
0,STT_GR_0001_0010_93600_to_106200,./downloads/STT_GR_0001_0010_93600_to_106200.wav,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...,12.600,STT_GR_0001,70-80__short__Teaching,70-80,short,Teaching,ཁོང་དེ་ནས་དགེ་འདུན་ཁྲི་ལ་སྔོན་ཟླ་རེ་བོད་གླེང་ད...,...,41,ཁོང། དེ་ནས་དགེ་དུས་ཁྲི་ལའི་རྔབ་བདག་རེད། བུ་གླ...,ཁོང་། དེ་ནས་དགེ་འདུན་ཁྲིི་ང་ད་རེད། བུ་གླིང་དེ...,ཁོང་་དེ་ས་དགེ་འདུལ་ཁྲིི་ལང་བདག་རེད། བུ་གླིང་དེ...,0.307263,0.262570,0.290503,57,49,54
1,STT_GR_0033_0105_224320_to_225856,./downloads/STT_GR_0033_0105_224320_to_225856.wav,https://d38pmlk0v88drf.cloudfront.net/wav16k/S...,1.536,STT_GR_0033,70-80__short__Teaching,70-80,short,Teaching,ཕ་གི་ཕར་རང་རེ་རྫས་ཚར་ལེ།,...,18,ཨ། ཕན་པ་འརེད་ཡོད་རེ། དེ་ལདྟ་ལུ།,ཕན་པ་ཡོད་ན་དེ་ལྟ་ལུ་,ཨ། ཕན་པ་ཡོད་ལ་དེ་ལྟ་ལུ།,0.833333,0.708333,0.750000,20,17,18


In [ ]:
df =  df['file_name', 'url', 'audio_duration', 'original_id', 'path', 'strata',
       'age_group', 'duration_category', 'content_type', 'uni',
       'base_transcript', 'base_cer', 'ft_v1_inference_checkpoint-5000',
       'ft_v1_cer_5000', 'ft_v1_inference_checkpoint-10000', 'ft_v1_cer_10000',
       'ft_v1_inference_checkpoint-19000', 'ft_v1_cer_19000',
       'ft_v2_inference_checkpoint-20000', 'ft_v2_cer_20000',
       'ft_v2_inference_checkpoint-32000', 'ft_v2_cer_32000',
       'ft_v2_inference_checkpoint-43000', 'ft_v2_cer_43000',
       'ft_v3_inference_checkpoint-1000', 'ft_v3_cer_1000',
       'ft_v3_inference_checkpoint-25000', 'ft_v3_cer_25000',
       'ft_v4_inference_checkpoint-22000', 'ft_v4_cer_22000',
       'ft_v5_base_inference_checkpoint-28000',
       'ft_v6_ft_inference_checkpoint-25000',
       'ft_v7_scratch_inference_checkpoint-23000', 'ft_v5_base_cer_28000',
       'ft_v6_ft_cer_25000', 'ft_v7_scratch_cer_23000', 'base_lev_dist',
       'ft_v1_lev_dist_5000', 'ft_v1_lev_dist_10000', 'ft_v1_lev_dist_19000',
       'ft_v2_lev_dist_20000', 'ft_v2_lev_dist_32000', 'ft_v2_lev_dist_43000',
       'ft_v3_lev_dist_1000', 'ft_v3_lev_dist_25000', 'ft_v4_lev_dist_22000',
       'ft_v5_base_lev_dist_28000', 'ft_v6_ft_lev_dist_25000',
       'ft_v7_scratch_lev_dist_23000']

In [13]:
df = df[
    # Basic file and audio information
    ['file_name', 'path', 'url', 'audio_duration', 'original_id', 
    
    # Metadata columns
    'strata', 'age_group', 'duration_category', 'content_type', 'uni',
    
    # Base model results
    'base_transcript', 'base_cer', 'base_lev_dist',
    
    # Model v1 results (grouped by checkpoint)
    'ft_v1_inference_checkpoint-5000', 'ft_v1_cer_5000', 'ft_v1_lev_dist_5000',
    'ft_v1_inference_checkpoint-10000', 'ft_v1_cer_10000', 'ft_v1_lev_dist_10000',
    'ft_v1_inference_checkpoint-19000', 'ft_v1_cer_19000', 'ft_v1_lev_dist_19000',
    
    # Model v2 results
    'ft_v2_inference_checkpoint-20000', 'ft_v2_cer_20000', 'ft_v2_lev_dist_20000',
    'ft_v2_inference_checkpoint-32000', 'ft_v2_cer_32000', 'ft_v2_lev_dist_32000',
    'ft_v2_inference_checkpoint-43000', 'ft_v2_cer_43000', 'ft_v2_lev_dist_43000',
    
    # Model v3 results
    'ft_v3_inference_checkpoint-1000', 'ft_v3_cer_1000', 'ft_v3_lev_dist_1000',
    'ft_v3_inference_checkpoint-25000', 'ft_v3_cer_25000', 'ft_v3_lev_dist_25000',
    
    # Model v4 results
    'ft_v4_inference_checkpoint-22000', 'ft_v4_cer_22000', 'ft_v4_lev_dist_22000',
    
    # Model v5 results
    'ft_v5_base_inference_checkpoint-28000', 'ft_v5_base_cer_28000', 'ft_v5_base_lev_dist_28000',
    
    # Model v6 results
    'ft_v6_ft_inference_checkpoint-25000', 'ft_v6_ft_cer_25000', 'ft_v6_ft_lev_dist_25000',
    
    # Model v7 results
    'ft_v7_scratch_inference_checkpoint-23000', 'ft_v7_scratch_cer_23000', 'ft_v7_scratch_lev_dist_23000',

    # Model v8 results
    'ft_v8_scartch_inference_checkpoint-10000', 'ft_v8_scratch_cer_10000', 'ft_v8_scratch_lev_dist_10000',
    'ft_v8_scartch_inference_checkpoint-55000', 'ft_v8_scratch_cer_55000', 'ft_v8_scratch_lev_dist_55000',
    'ft_v8_scartch_inference_checkpoint-59000', 'ft_v8_scratch_cer_59000', 'ft_v8_scratch_lev_dist_59000',
    ]
]

In [ ]:
!pip install ipywidgets

In [ ]:
import pandas as pd
from datasets import Dataset
from huggingface_hub import notebook_login
import os

# First login to Hugging Face
notebook_login()


In [14]:


# Convert pandas DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Push to the Hugging Face Hub
dataset.push_to_hub(
    "ganga4364/garchen_rinpoche_evaluation_results",  # Change to your desired repo name
    private=False,  # Set to True if you want a private dataset
    token=None,  # Will use the token from the login step
    commit_message="Upload Garchen Rinpoche ASR evaluation results"
)

print("Dataset successfully pushed to Hugging Face Hub!")


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

                                        :  44%|####3     |  533kB / 1.21MB            

README.md: 0.00B [00:00, ?B/s]

Dataset successfully pushed to Hugging Face Hub!
